In [313]:
import pymongo
client = pymongo.MongoClient(host='120.79.139.239', port=27017)
db = client['weibo']

# User Collection

In [ ]:
import re, sys, bson
collection = db['users']
regex = re.compile('(?:谈股|财经|说股|期货|外汇|讲股|论股|看股|港股|美股|理财师|A股|经济政策|短线|炒股|荐股|盈亏|炒股|股利|股海|股民|股票|个股|做空|诱多|诱空|踏空|长空|短空|长多|短多|死多|翻多|翻空|多杀多|扎空|空仓|建仓|满仓|斩仓|减仓|加仓|重仓|清仓|套牢|补仓|仓位|庄家|震仓|追涨|杀跌|止盈|抄底|逃顶|盘整|回档|坐庄|吸筹|对敲|洗盘|散户|中户|坐轿|抬轿|筹码|抢帽子|多头陷阱|空头陷阱|护盘|跳空|开盘价|收盘价|最高价|最低价|成交量|放量|缩量|热门股|冷门股|白马股|黑马股|龙头股|阴跌|换手率|现手|平开|低开|高开|内盘|外盘|均价|浮筹|市盈率|含权|回购|基本面分析|量比|每股|收益|市净率|探底|填权|停牌|退市)')    
nregex = re.compile('(?:公司|企业|CEO|基金会|大学|学院|政府)')

weibo_list = collection.find({}, {'id': 1, 'description': 1})

In [ ]:
print (weibo_list.count())

In [ ]:
id_del = []
for x in weibo_list:
    desp = x['description']
    if re.search(nregex, desp) != None or re.search(regex, desp) == None:
        id_del.append(x['id'])

for id in id_del:
    collection.delete_one({'id': id})

In [ ]:
results = collection.find({}, {'id': 1})
id_list = []
for result in results:
    id_list.append(str(result['id']))
print (len(id_list))

In [2]:
collection = db['weibos']
results = collection.find({}, {'created_at': 1})
time_list = []
import re
for result in results:
    if re.search('2018-07-20', result['created_at']) != None:
        time_list.append(str(result['created_at']))
time_list = sorted(time_list, reverse=True)
print (len(time_list))
print (time_list)

2007
['2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00

False

In [317]:
import time
date = time.strftime('%Y-%m-%d', time.localtime())
ldate = time.strftime('%Y-%m-%d', time.localtime(time.time() - 24 * 60 * 60))
lldate = time.strftime('%Y-%m-%d', time.localtime(time.time() - 2 * 24 * 60 * 60))
print (date)
print (ldate)
print (lldate)
h = "{}|{}|{}".format(date, ldate, lldate)
results1 = collection.find({'created_date': {'$regex': date}})
results2 = collection.find({'created_date': {'$regex': ldate}})
print (results1.count())
print (results2.count())

# weibo_id_list = []
# for delete in deletes: 
#     weibo_id_list.append(delete['id'])
# print (len(weibo_id_list))

2018-07-23
2018-07-22
2018-07-21
886
1683


/home/impulse/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  # This is added back by InteractiveShellApp.init_path()
/home/impulse/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  if sys.path[0] == '':


In [ ]:
print (weibo_id_list)

# Analyzing

In [125]:
import pymongo
import pandas as pd
def get_data():
    client = pymongo.MongoClient(host='120.79.139.239', port=27017)
    db = client['weibo']
    collection = db['weibos']
    results = collection.find({}, {'user':1,'attitudes_count':1,'comments_count':1,'reposts_count':1,'text':1,'full_text':1})
    data = pd.DataFrame(list(results))
    if 'attitudes_count' in data.columns: ## if it is normal
        data = data.get(['user','attitudes_count','comments_count','reposts_count','text', 'full_text'])
        data.columns = ['user', 'at_cnt', 'cmt_cnt', 'rep_cnt', 'text', 'f_text']
        for i in range(len(data)):
            if len(str(data.loc[i, 'f_text'])) > 135:
                data.loc[i, 'text'] = data.loc[i, 'f_text']
        data = data.drop('f_text', axis=1)
    else:
        print ('some errors occur')
        
    return data

In [268]:
import re

def read_stop_words(): 
    stop_words = []
    stop_letters = []
    with open('stop_words.txt') as sp:
        for x in sp:
            stop_words.append(x[:-1])
    with open('stop_letters.txt') as sp:
        for x in sp:
            stop_letters.append(x[:-1])
    stop_words = set(stop_words)
    re_stop_letter = re.compile('|'.join(stop_letters))
    return (stop_words, re_stop_letter)

In [267]:
import re, jieba
jieba.load_userdict('jieba_dict_companys')

def get_clean_text(data, output=True):
    texts = []
    for cnt, raw in enumerate(data.text):
        del_name = re.findall('@([^ |<|:|\(|\)|\\|\/|<|>|\[|\]]+)', raw)
        for name in del_name:
            raw = re.sub(name, '', raw)
        raw = re.sub('<br />', '，', raw)
        raw = re.sub('[^\u4e00-\u9fa5|，|!|。]+|\?{1}|:{1}', '', raw)
        raw = re.sub(' +', ' ', raw)
        raw = re.sub('，+', '，', raw)
        raw = re.sub('转发微博|全文', '', raw)
        texts.append(raw)
        
        if cnt % 100 == 0 and output == True:
            print ('cnt = {}'.format(cnt))
    if len(texts) == len(data):
        return pd.Series(texts)
    else:
        print('not the same length')
        return None

In [269]:
import re, jieba
import jieba.posseg as postag
                           
def get_words(data, re_stop_letter, stop_words, output=True):
    words = []
    for cnt, raw in enumerate(data.c_text):
        result = postag.cut(raw)
        raw = [x.word for x in result if (len(x.word) > 1 and 'n' in x.flag and re.search(re_stop_letter, x.word) == None and x.word not in stop_words)]
        words.append(raw)
        
        if cnt % 100 == 0 and output == True:
            print ('cnt = {}'.format(cnt))
            
    return pd.Series(words)

In [270]:
def get_word_count(data):
    se = data.words
    count = {}
    for lst in se:
        for x in lst:
            if x in count:
                count[x] += 1
            else:
                count[x] = 1
    return count

In [126]:
data = get_data()

In [276]:
stop_words, re_stop_letter = read_stop_words()

In [277]:
data['c_text'] = get_clean_text(data)

cnt = 0
cnt = 100
cnt = 200
cnt = 300
cnt = 400
cnt = 500
cnt = 600
cnt = 700
cnt = 800
cnt = 900
cnt = 1000
cnt = 1100
cnt = 1200
cnt = 1300
cnt = 1400
cnt = 1500
cnt = 1600
cnt = 1700
cnt = 1800
cnt = 1900
cnt = 2000
cnt = 2100
cnt = 2200
cnt = 2300
cnt = 2400
cnt = 2500
cnt = 2600
cnt = 2700
cnt = 2800
cnt = 2900
cnt = 3000
cnt = 3100
cnt = 3200
cnt = 3300
cnt = 3400
cnt = 3500
cnt = 3600
cnt = 3700
cnt = 3800
cnt = 3900
cnt = 4000
cnt = 4100
cnt = 4200
cnt = 4300
cnt = 4400
cnt = 4500
cnt = 4600
cnt = 4700
cnt = 4800
cnt = 4900
cnt = 5000
cnt = 5100
cnt = 5200


In [280]:
data['words'] = get_words(data=data, re_stop_letter=re_stop_letter, stop_words=stop_words)

cnt = 0
cnt = 100
cnt = 200
cnt = 300
cnt = 400
cnt = 500
cnt = 600
cnt = 700
cnt = 800
cnt = 900
cnt = 1000
cnt = 1100
cnt = 1200
cnt = 1300
cnt = 1400
cnt = 1500
cnt = 1600
cnt = 1700
cnt = 1800
cnt = 1900
cnt = 2000
cnt = 2100
cnt = 2200
cnt = 2300
cnt = 2400
cnt = 2500
cnt = 2600
cnt = 2700
cnt = 2800
cnt = 2900
cnt = 3000
cnt = 3100
cnt = 3200
cnt = 3300
cnt = 3400
cnt = 3500
cnt = 3600
cnt = 3700
cnt = 3800
cnt = 3900
cnt = 4000
cnt = 4100
cnt = 4200
cnt = 4300
cnt = 4400
cnt = 4500
cnt = 4600
cnt = 4700
cnt = 4800
cnt = 4900
cnt = 5000
cnt = 5100
cnt = 5200


In [281]:
count = get_word_count(data)

In [282]:
vocab = []
for k, v in count.items():
    vocab.append((k,v))
sorted_vocab = list(sorted(vocab, key = lambda v : v[1], reverse = 1))

In [283]:
for x in sorted_vocab[:100]:
    print (x)

('疫苗', 3792)
('长生', 1704)
('生产', 906)
('投资', 524)
('金融', 442)
('销售', 396)
('链接', 375)
('上证指数', 372)
('国家药监局', 370)
('头条', 340)
('监管', 328)
('创业板', 307)
('基金', 292)
('医药', 276)
('狂犬病', 271)
('李克强', 264)
('国家', 260)
('概念股', 251)
('牛股', 242)
('直播', 239)
('股东', 238)
('软件', 213)
('上市', 210)
('药品', 197)
('交代', 194)
('军工', 189)
('管理', 188)
('强势', 187)
('违法', 172)
('资产', 172)
('上市公司', 169)
('小米', 168)
('杠杆', 167)
('新浪', 165)
('石墨', 165)
('合格', 160)
('黄金', 159)
('康泰', 155)
('立案', 153)
('涨幅', 153)
('集体', 148)
('落地', 145)
('调整', 144)
('底线', 142)
('龙头', 142)
('变化', 142)
('均线', 141)
('营销', 134)
('证券', 131)
('彻查', 130)
('工业', 130)
('检验', 127)
('制度', 127)
('进口', 126)
('道德', 125)
('工作', 125)
('子公司', 123)
('国产', 122)
('狂犬', 120)
('电解锰', 120)
('细则', 119)
('科技股', 118)
('流向', 117)
('茅台', 117)
('销售费用', 116)
('概率', 116)
('启动', 113)
('消息面', 113)
('腾讯', 112)
('特朗普', 112)
('品种', 111)
('中标', 110)
('流动性', 109)
('检查', 106)
('规范', 105)
('金融股', 105)
('信任', 104)
('政府', 104)
('总理', 101)
('估值', 101)
('调查组', 98)
('新材',

# sentiments

In [286]:
import requests, re, time, json

def get_sentiment(data, output=True):
    headers = {
        'Content-Type': 'application/json'
    }
    params = {
        'access_token': '24.cb5b8cccad6a49c21d4cccd1a047f9ae.2592000.1534940191.282335-11569351'
    }
    positive_prob = []
    for cnt, text in enumerate(data.c_text):
        start_time = time.clock()
        post_json = {
            "text": text
        }

        response = requests.post('https://aip.baidubce.com/rpc/2.0/nlp/v1/sentiment_classify',
                                params=params, headers=headers, json=post_json)
        if response.status_code != 200:
            time.sleep(2.0)
            response = requests.post('https://aip.baidubce.com/rpc/2.0/nlp/v1/sentiment_classify',
                                params=params, headers=headers, json=post_json)

        if response.text != None:
            res_json = json.loads(response.text)
            if res_json.get('items') != None and res_json.get('items')[0].get('positive_prob') != None: 
                prob = res_json.get('items')[0].get('positive_prob')
                #print (prob)
                positive_prob.append(prob)
            else:
                print('-1')
                positive_prob.append('-1')
        else:
            print('-1')
            positive_prob.append('-1')
        elapsed = (time.clock() - start_time)
        time.sleep(0.25 - elapsed)

        if cnt % 100 == 0 and output:
            print ('cnt = {}'.format(cnt))
    if len(data) != len(positive_prob):
        print ('length not equal')
        
    return pd.Series(positive_prob)
    

In [287]:
data['senti'] = get_sentiment(data)

cnt = 0
-1
-1
cnt = 100
cnt = 200
cnt = 300
cnt = 400
cnt = 500
-1
-1
cnt = 600
cnt = 700
-1
-1
-1
cnt = 800
-1
cnt = 900
cnt = 1000
cnt = 1100
cnt = 1200
cnt = 1300
-1
-1
cnt = 1400
cnt = 1500
-1
cnt = 1600
cnt = 1700
cnt = 1800
cnt = 1900
cnt = 2000
cnt = 2100
cnt = 2200
cnt = 2300
-1
-1
cnt = 2400
-1
cnt = 2500
-1
cnt = 2600
-1
cnt = 2700
-1
cnt = 2800
-1
cnt = 2900
cnt = 3000
-1
cnt = 3100
cnt = 3200
-1
cnt = 3300
cnt = 3400
cnt = 3500
-1
-1
-1
cnt = 3600
cnt = 3700
-1
cnt = 3800
cnt = 3900
cnt = 4000
cnt = 4100
cnt = 4200
cnt = 4300
-1
cnt = 4400
-1
cnt = 4500
cnt = 4600
-1
cnt = 4700
-1
cnt = 4800
cnt = 4900
-1
cnt = 5000
cnt = 5100
cnt = 5200


In [288]:
print (data.head())
data.to_csv('weibo_test_data.csv')

         user  at_cnt  cmt_cnt  rep_cnt  \
0  3146057615       0        0        0   
1  2280207591       0        0        0   
2  2280207591       4        1        0   
3  2280207591      97        5        2   
4  2280207591      21        7        0   

                                                text  \
0  给各位下周几点意见： 1、监管松绑，短期是否能带来增量资金回流的信心，下周见分晓，光靠大机构...   
1  转发微博#RisaBeira粉丝福利##云南店4周年店庆#四年来，我们为3916位新娘献上嫁...   
2  新舞林大会毛晓彤跳牛仔舞<br />当红小花加盟《新舞林大会》，新舞林大会毛晓彤跳牛仔舞，超...   
3  林俊杰歌迷车祸，泪崩。小时候听说这是一条通往天堂的路。 <a data-url="http:...   
4  想带你去看晴空万里。 <a data-url="http://t.cn/Rg3R5Ix" h...   

                                               words  \
0                         [监管, 蓝筹权, 连阳, 均线, 创业板, 刚性]   
1  [店庆, 新娘, 嫁衣, 新娘, 邀请函, 晚八点, 复古, 音响, 口红, 心店, 婚纱,...   
2  [舞林, 毛晓彤, 牛仔, 当红, 小花, 加盟, 舞林, 舞林, 毛晓彤, 牛仔, 精彩,...   
3                         [林俊杰, 歌迷, 车祸, 泪崩, 小时候, 天堂]   
4                                                 []   

                                              c_text     senti  
0 